In [5]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import Word2Vec
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import *

# https://stackoverflow.com/questions/57451719/since-spark-2-3-the-queries-from-raw-json-csv-files-are-disallowed-when-the-ref
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

FILE_NAME = "tweets.json"

original_data = spark.read.option("multiline","true").json(FILE_NAME).cache()

selectId = "480875170"

# Q1

# find all replies
user_replyTo = original_data.select("user_id", "replyto_id").where("replyto_id is not null")
# find all retweets
user_retweet = original_data.select("user_id", "retweet_id").where("retweet_id is not null")

# combine two data:(user_id, relatedTweetId)
user_combine = user_retweet.union(user_replyTo)
user_data = user_combine.rdd.map(lambda row:(row[0], str(row[1]))).groupByKey().mapValues(list)

def changeToString(row):
    user_id = row[0]
    data = " ".join(row[1])
    return (user_id, data)
    
    
user_tweet_string = user_data.map(changeToString)

train_data = user_tweet_string.toDF(["user_id", "data"])

# idf train data
# https://spark.apache.org/docs/latest/ml-features.html#tf-idf
tokenizer = Tokenizer(inputCol="data", outputCol="words")
wordsData = tokenizer.transform(train_data)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures",numFeatures=131072)
featurizedData = hashingTF.transform(wordsData)
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

# drop useless columns
clearData = rescaledData.drop("data").drop("words").drop("rawFeatures")

target_user = clearData.filter("user_id =" + selectId).collect()[0][1]
other_user = clearData.filter("user_id !=" + selectId).rdd.map(lambda row : (row[0], row[1]))

def cosine(a,b):
    return a.dot(b) / (a.norm(2) * b.norm(2))

found_user = other_user.map(lambda row:(row[0], cosine(target_user, row[1])))
print("Top five users with similar interest based on TF-IDF:")
print(found_user.sortBy(lambda row:row[1], ascending=False).take(5))


# Word2Vec
text_data = user_tweet_string.toDF(["user_id", "text"])
word_data = text_data.select('user_id', split("text",' ').alias("words"))
# word_data.show(100, truncate=False)

# train data
word2Vec = Word2Vec(vectorSize=300, minCount=1, inputCol="words", outputCol="vector")
model = word2Vec.fit(word_data)
result = model.transform(word_data).drop("words")

target_user_1 = result.filter("user_id =" + selectId).collect()[0][1]
other_user_1 = result.filter("user_id !=" + selectId).rdd.map(lambda row : (row[0], row[1]))
found_user_1 = other_user_1.map(lambda row:(row[0], cosine(target_user_1, row[1])))

print("Top five users with similar interest based on Word2Vec:")
print(found_user_1.sortBy(lambda row:row[1], ascending=False).take(5))


# get user_id 
user_ids = original_data.select("user_id").rdd.map(lambda row:row[0]).distinct().collect()

# create dictionary for user_id
user_id_mapper = dict()
for user_id in user_ids:
    user_id_mapper[user_id] = len(user_id_mapper)

def extract_id(rows):
    return [(mention_user[0]) for mention_user in rows[0]]

# extract mentioned user id
mention_user_info = original_data.select("user_mentions").where("user_mentions is not null")
mention_user_ids = mention_user_info.rdd.flatMap(extract_id).distinct().collect()
# create dictionary for mentioned user id
mention_user_id_mapper = dict()
for mention_user_id in mention_user_ids:
    mention_user_id_mapper[mention_user_id] = len(mention_user_id_mapper)

# sc.broadcast(user_id_mapper)
# sc.broadcast(mention_user_id_mapper)

def extract_data(rows):
    user_id = rows[0]
    return [(user_id, mention_user_id_mapper[mention_user[0]], 1) for mention_user in rows[1]]

def create_key(rows):
    user_id = rows[0] 
    mention_user = rows[1] 
    counter = rows[2]
    return ((user_id,mention_user), counter)

mention_user = original_data.select("user_id","user_mentions").where("user_mentions is not null")
# extract data
clear_data = mention_user.rdd.map(lambda row:(user_id_mapper[row[0]],row[1])).flatMap(extract_data)
# combine user_id mention_user_id as a key
key_data = clear_data.map(create_key)
# calculate                    
clear_key = key_data.reduceByKey(lambda a,b : a + b).map(lambda row : (row[0][0], row[0][1], row[1]))
train_data = clear_key.toDF(["user_id", "mention_user_id", "counter"])


# indexer_1 = StringIndexer(inputCol="user_id", outputCol="user_id_Index")
# indexed_1 = indexer_1.fit(train_data).transform(train_data)

# indexer_2 = StringIndexer(inputCol="mention_user_id", outputCol="mention_user_id_Index")
# indexed_2 = indexer_2.fit(indexed_1).transform(indexed_1)

# a_b = indexed_2.drop("user_id").drop("mention_user_id")
# re_format = a_b[["user_id_Index","mention_user_id_Index","counter"]]

# https://spark.apache.org/docs/latest/ml-collaborative-filtering.html
# train data
als = ALS(userCol="user_id", itemCol="mention_user_id", ratingCol="counter",coldStartStrategy="drop")
model = als.fit(train_data)
model_recommend = model.recommendForAllUsers(5).collect()

# converter = IndexToString(inputCol="user_id_Index", outputCol="raw_user_id_Index")
# converted = converter.transform(model_recommend)
# converted.show()

print("user recomendation is:")
for rows in model_recommend:
    print(user_ids[rows[0]], ":", end=" ")
    for data in rows[1]:
        print(mention_user_ids[data[0]], end="   ")
    print()

sc.stop()
# for rows in model_recommend:
#     if user_ids[rows[0]] == 1242709697397952512:
#         print(user_ids[rows[0]], ":", end=" ")
#         for data in rows[1]:
#             print(mention_user_ids[data[0]], end="   ")


Top five users with similar interest based on TF-IDF:
[(1012137970885906432, 0.43845261466356117), (282643059, 0.4010760488191615), (138309569, 0.39620107225594015), (545185861, 0.38650057152931333), (848329022211620864, 0.3659575900891094)]
Top five users with similar interest based on Word2Vec:
[(1323090359581200385, 0.8946106656633247), (875668452463259648, 0.874272407624937), (1263371505733218304, 0.874272407624937), (581812461, 0.874272407624937), (546664702, 0.8634570290319374)]
user recomendation is:
17375281 : 111404710   38400130   19898168   1326561978308059137   3044682647   
56722736 : 24259259   346102488   1244013067790364673   3094649957   807095   
866949673461981184 : 56488059   20457806   2374745089   7258242   1652541   
776625931234390017 : 24259259   346102488   1244013067790364673   3094649957   807095   
1220079661201399809 : 380648579   100248020   32828872   1656958189   16973333   
107548394 : 24259259   26574283   216294504   2791419078   185025785   
1484909

272087341 : 24259259   5392522   71372467   1643123766   149075155   
13440002 : 24259259   346102488   1244013067790364673   3094649957   807095   
55190116 : 20457806   56488059   29780473   299273962   18831926   
2440319112 : 26351144   17674244   1643123766   20578408   22053725   
396293876 : 17154865   145560988   842678603305377793   83036348   621064274   
232086612 : 24259259   346102488   1244013067790364673   3094649957   807095   
66796845 : 346102488   1244013067790364673   15012486   247205027   15864446   
299539882 : 637196658   56488059   20457806   257152792   39879120   
282116142 : 24259259   346102488   1244013067790364673   3094649957   807095   
1709590752 : 24259259   346102488   1244013067790364673   3094649957   807095   
1282841766362861568 : 24259259   26574283   216294504   2791419078   185025785   
3094649957 : 24259259   3094649957   842678603305377793   1917731   346102488   
30874049 : 24259259   346102488   1244013067790364673   3094649957   807095   

3220831538 : 24259259   5392522   71372467   1643123766   149075155   
517212934 : 380648579   100248020   32828872   1656958189   16973333   
54979466 : 24259259   346102488   1244013067790364673   3094649957   21802625   
289239316 : 24259259   26574283   216294504   2791419078   185025785   
903261743132102656 : 15859912   1353769946556325889   19881665   16422863   330574021   
870150798017150976 : 24259259   346102488   1244013067790364673   3094649957   807095   
47771310 : 24259259   346102488   1244013067790364673   3094649957   807095   
1350324267779579905 : 1003107003693137921   2876041031   866065134   842678603305377793   20457806   
784881611422703616 : 1104447042993164290   346102488   25026558   24259259   1244013067790364673   
196860724 : 24259259   26574283   216294504   2791419078   185025785   
996495620025307136 : 14335586   347468570   9814812   1331294857625624576   141468289   
402983524 : 24259259   26574283   216294504   2791419078   185025785   
2848802507 :

220795502 : 380648579   100248020   32828872   1656958189   16973333   
1370970455394373636 : 24259259   346102488   1244013067790364673   3094649957   807095   
741088820053999616 : 9814812   20457806   56488059   2318270108   15115280   
287157353 : 24259259   26574283   216294504   2791419078   185025785   
173417120 : 24259259   346102488   1244013067790364673   3094649957   807095   
51709107 : 320432933   281877818   61863570   80102969   2467791   
153796552 : 24259259   346102488   1244013067790364673   3094649957   807095   
1086931189380788225 : 56488059   20457806   14296273   59159771   33960482   
1810483688 : 24259259   346102488   1244013067790364673   3094649957   807095   
3741293727 : 24259259   346102488   1244013067790364673   3094649957   807095   
539969895 : 24259259   26574283   216294504   2791419078   185025785   
66753218 : 24259259   346102488   1244013067790364673   3094649957   807095   
731249725 : 380648579   100248020   32828872   1656958189   16973333 

101367781 : 24259259   346102488   1244013067790364673   3094649957   807095   
21919110 : 191807697   1599212660   56488059   20457806   221718444   
15234684 : 24259259   346102488   1244013067790364673   3094649957   807095   
1231757546043695104 : 24259259   346102488   1244013067790364673   3094649957   807095   
1267255422798442496 : 20457806   56488059   1349149096909668363   804414007377416192   1026461628336926720   
1250953106742886400 : 24259259   346102488   1244013067790364673   3094649957   807095   
1153952256305324032 : 24259259   83440337   260371201   842678603305377793   860654500431880192   
1296037183 : 24259259   346102488   1244013067790364673   3094649957   807095   
367300545 : 380648579   100248020   32828872   1656958189   16973333   
54999242 : 191807697   1599212660   56488059   20457806   221718444   
115978171 : 24259259   26574283   216294504   2791419078   185025785   
3156951007 : 20457806   56488059   153944899   9814812   1385797144209096705   
22198

...

257121078 : 24259259   26574283   216294504   2791419078   185025785   
193238639 : 24259259   346102488   1244013067790364673   3094649957   807095   
29693196 : 24259259   346102488   1244013067790364673   3094649957   807095   
35667848 : 24259259   2729061   1042116448372580352   2985479932   927968133251518464   
422789312 : 18831926   939091   803694179079458816   14147716   20457806   
863073687855149056 : 24259259   26574283   216294504   2791419078   185025785   
1187912743761960960 : 24259259   26574283   216294504   2791419078   185025785   
997097702172385280 : 191807697   1599212660   56488059   20457806   221718444   
17158238 : 24259259   26574283   216294504   2791419078   185025785   
214270722 : 24259259   346102488   1244013067790364673   3094649957   807095   
1369285800 : 149075155   449027230   15859912   2420618906   1348750216225296384   
813057413742034944 : 24259259   3290364847   32840323   179461986   132630502   
3895800435 : 627341141   1562961949   119363

964585706 : 24259259   346102488   1244013067790364673   3094649957   807095   
712673449 : 24259259   2195885540   842678603305377793   369534928   854725669   
823677630 : 133081348   408793153   1267708379138535424   299273962   22429979   
2421647559 : 24259259   26574283   216294504   2791419078   185025785   
262649252 : 24259259   346102488   1244013067790364673   3094649957   807095   
864258388414398464 : 380648579   100248020   32828872   1656958189   16973333   
152728674 : 24259259   26574283   216294504   2791419078   185025785   
1213653590402748419 : 380648579   100248020   32828872   1656958189   16973333   
178804952 : 24259259   26574283   216294504   2791419078   185025785   
529008588 : 24259259   32840323   842678603305377793   837335683215798272   3290364847   
190519323 : 24259259   26574283   216294504   2791419078   185025785   
50613 : 380648579   100248020   32828872   1656958189   16973333   
1242673974917677056 : 24259259   346102488   1244013067790364673  

19676102 : 237341484   1278694316752490500   881974187589603328   807357676300730368   191807697   
879996270617567232 : 24259259   26574283   216294504   2791419078   185025785   
1127196523287199746 : 22053725   854725669   1311714648622870529   1171155332917321730   17674244   
1273431872262098944 : 28838219   9814812   865004396681207809   881974187589603328   1278694316752490500   
871308968420704258 : 24259259   346102488   1244013067790364673   3094649957   807095   
1098955281428557829 : 24259259   13393052   346102488   1244013067790364673   18831926   
1436401104 : 24259259   148529707   1059652318285709312   26574283   185025785   
707001718534492160 : 133081348   408793153   1267708379138535424   299273962   22429979   
1241849511137259520 : 24259259   346102488   1244013067790364673   3094649957   807095   
78516144 : 24259259   346102488   1244013067790364673   3094649957   807095   
177034593 : 24259259   26574283   216294504   2791419078   185025785   
17845089 : 380648

1159770232925409281 : 380648579   100248020   32828872   1656958189   16973333   
3465954552 : 24259259   26574283   216294504   2791419078   185025785   
120839423 : 24259259   26574283   216294504   2791419078   185025785   
263607988 : 20457806   56488059   15115280   224329419   9814812   
74337616 : 24259259   346102488   1244013067790364673   3094649957   807095   
825718810258898946 : 191807697   1599212660   56488059   20457806   221718444   
356411775 : 20457806   56488059   15115280   224329419   9814812   
991755746 : 24259259   346102488   1244013067790364673   3094649957   807095   
135487556 : 24259259   346102488   1244013067790364673   3094649957   807095   
2678984257 : 24259259   346102488   1244013067790364673   3094649957   807095   
62304590 : 24259259   13393052   346102488   1244013067790364673   18831926   
24431674 : 842678603305377793   17674244   594028334   408793153   1267708379138535424   
1353149964994535424 : 169963553   36913032   18750999   79815574497

2746728698 : 24259259   346102488   1244013067790364673   3094649957   807095   
4162785695 : 348687298   55060090   65201417   13049362   271319718   
554841435 : 24259259   346102488   1244013067790364673   3094649957   807095   
800380614 : 24259259   346102488   1244013067790364673   3094649957   807095   
549572614 : 24259259   346102488   1244013067790364673   3094649957   807095   
97611405 : 24259259   346102488   1244013067790364673   3094649957   21802625   
1172978680710066176 : 24259259   26574283   216294504   2791419078   185025785   
113487920 : 24259259   346102488   1244013067790364673   3094649957   807095   
289452611 : 24259259   346102488   1244013067790364673   3094649957   807095   
47530811 : 24259259   26574283   216294504   2791419078   185025785   
322806753 : 24259259   83440337   260371201   842678603305377793   860654500431880192   
1621469700 : 24259259   346102488   1244013067790364673   3094649957   807095   
1122207470640881664 : 24259259   346102488  